<a href="https://colab.research.google.com/github/Existanze54/sirius-neural-networks-2024/blob/main/Practices/07S_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказание сайтов сплайсинга

Сплайсинг $-$ происходящее в ходе процессинга РНК вырезание определённых нуклеотидных последовательностей (интронов) из молекул РНК и соединения оставшихся последовательностей, сохраняющихся в «зрелой» молекуле (экзонов).
Участвующие в процессе сплайсинга белки распознают границы интронов благодаря наличию в последовательности донорных (5') и акцепторных (3') сайтов.

Уже знакомый нам датасет из <a href="https://www.kaggle.com/muhammetvarl/splicejunction-gene-sequences-dataset">Kaggle</a> содержит закодированные последовательности РНК, размеченные на 3 класса: содержит донорный сайт, содержит акцепторный сайт, и не содержит таких сайтов. Воспользуемся им для изучения возможностей RNN.


<img src="https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/splicing.gif" alt="Drawing" width= "500px;"/>

Датасет представлен закодированными в бинарном формате последовательностями длиной 60, по три "признака" на нуклеотид (всего 180 фичей):

* A $\rightarrow$ 1 0 0
* C $\rightarrow$ 0 1 0
* G $\rightarrow$ 0 0 1
* U $\rightarrow$ 0 0 0

Так можно делать, но torch позволяет нам делать хитрее. Для каждого из трех "признаков" отведем свою размерность, таким образом сведя датасет $N * 180$ к $N * 60 * 3$.

Загрузим датасет:

In [ ]:
import pandas as pd
import torch

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
!if [ ! -f ./rna.csv ]; then wget https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/rna.csv; fi

--2022-12-14 16:15:45--  https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/rna.csv
Resolving kodomo.fbb.msu.ru (kodomo.fbb.msu.ru)... 93.180.63.127
Connecting to kodomo.fbb.msu.ru (kodomo.fbb.msu.ru)|93.180.63.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1154490 (1.1M) [text/csv]
Saving to: ‘rna.csv’

rna.csv             100%[===================>]   1.10M   956KB/s    in 1.2s    

2022-12-14 16:15:49 (956 KB/s) - ‘rna.csv’ saved [1154490/1154490]



In [ ]:
df = pd.read_csv("rna.csv")

print(df.shape)
df.head()

(3186, 181)


,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A29,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,...,A141,A142,A143,A144,A145,A146,A147,A148,A149,A150,A151,A152,A153,A154,A155,A156,A157,A158,A159,A160,A161,A162,A163,A164,A165,A166,A167,A168,A169,A170,A171,A172,A173,A174,A175,A176,A177,A178,A179,class
0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,...,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,3
1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,3
2,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,0,1,...,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,3
3,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1
4,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,2


In [ ]:
df["class"].value_counts()

3    1654
1     767
2     765
Name: class, dtype: int64

Приведем метки классов к нормальному виду:

In [ ]:
y = df["class"]
X = df.drop(['class'], axis=1)

from sklearn.preprocessing import LabelEncoder
# Encode class values as integers and perform label-encoding
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
y

array([2, 2, 2, ..., 1, 0, 0])

In [ ]:
encoder.classes_

## Задание 1

Поменяйте размерность датасета: от $N * 180$ к $N * 3 * 60$. Вам поможет <a href='https://pytorch.org/docs/stable/generated/torch.Tensor.reshape.html'>torch.Tensor.reshape<a/>.

In [ ]:
# Train-Test
from sklearn.model_selection import train_test_split
# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=0)

# torch.tensor infers the dtype automatically, while torch.Tensor returns a torch.FloatTensor.
X_train = torch.Tensor(X_train.values)
X_train = X_train.reshape((#put your code here))
y_train = torch.Tensor(y_train)
y_train = y_train.type(torch.LongTensor)

X_test = torch.Tensor(X_test.values)

X_test = X_test.reshape((#put your code here))

y_test = torch.Tensor(y_test)
y_test = y_test.type(torch.LongTensor)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## Задание 2

Реализуйте класс spliceJuncPredictor. Нейросеть должна содержать 1 рекуррентный слой и 1 полносвязный. Укажите batch_first=True.  

In [ ]:
import torch
import torch.nn as nn

In [ ]:
?nn.RNN

In [ ]:
class spliceJuncPredictor(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn = #put your code here
        self.fc = #put your code here

    def forward(self, x):
        out, h = self.rnn(x)
        y = self.fc(h)
        return y[0], h # убираем "лишнюю" размерность

Проверьте, что сеть правильно работает и возвращает тензор нужной размерности (*какой?*). Размерность скрытого слоя можете подобрать по желанию.

In [ ]:
input_size = 3
hidden_size = 16
rnn = spliceJuncPredictor(input_size, hidden_size)

out, h = rnn(X_train)
print(out.shape)

torch.Size([2230, 3])


Воспользуйтесть кодом и обучите сеть. Оцените качество пресказания.

In [ ]:
import torch.utils.data as data_utils

EPOCHS_NUM = 150
BATCH_SIZE = 400

train_loader = data_utils.DataLoader(data_utils.TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
test_loader = data_utils.DataLoader(data_utils.TensorDataset(X_test, y_test), batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
def validate(model,testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in testloader:
            outputs, h = model(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

def train(model, num_epochs, learning_rate = 0.01):

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # Train the model
    model.train()
    for epoch in range(num_epochs):
        for feat_batch, labels_batch in train_loader:
            optimizer.zero_grad()
            outputs, h = model(feat_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()
        accuracy = validate(model,test_loader)
        if epoch % 25 == 0:
            print(f"Epoch {epoch} Loss {loss.item():.2f} Accuracy {accuracy:.2f}")

In [ ]:
rnn = spliceJuncPredictor(input_size, hidden_size)
train(rnn, num_epochs=EPOCHS_NUM)

In [ ]:
from sklearn.metrics import classification_report

def get_report(model, X_test, y_test):
    model.eval()
    outputs, h = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    model_report = classification_report(y_test, predicted)
    print(model_report)

In [ ]:
get_report(rnn, X_test, y_test)

## Задание 3

Замените RNN слой на <a href="https://pytorch.org/docs/stable/generated/torch.nn.GRU.html">GRU</a> или <a href="https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html">LSTM</a> (на выбор). Внесите необходимые изменения. Обучите сеть, оцените качество предсказания.

In [ ]:
#put your code here

In [ ]:
#put your code here

# Генерация изображений

Попробуем сгенерировать новые изображения, обучив простую генеративно-состязательную сеть на датасете [FashionMNIST](https://www.kaggle.com/zalando-research/fashionmnist) -- датасете изображений предметов одежды размером 28*28.

In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [ ]:
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import FashionMNIST

#Load the data

mnist = FashionMNIST(root='data',
              train=True,
              download=True,
              transform=Compose([ToTensor(), Normalize(mean=(0.5,), std=(0.5,))]))

In [ ]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

img, label = mnist[0]
print('Label: ', label)
img_norm = denorm(img)
plt.imshow(img_norm[0], cmap='gray')

In [ ]:
data_loader = torch.utils.data.DataLoader(mnist, batch_size=64, shuffle=True)

## Задание 4.

Напишите полносвязный cGAN. В данном случае условие это lablel (номер класса) предмета одежды.

1. Сделайте эмбеддинг для лэйблов внутри модели
2. С помощью torch.cat добавьте этот эмбеддинг ко входу генератора и дискриминатора. Входное латентное пространство зададим, например, размера 100.

Ниже приведены рекомендованные для этого задания архитектуры сетей. Используйте их, вносите изменения по желанию.

3. Используйте следующую для генератора:
        self.model = nn.Sequential(
            nn.Linear(?, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )
        
4. Используйте такую архитектуру для дискриминатора:
        self.model = nn.Sequential(
            nn.Linear(?, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        self.label_emb = nn.Embedding(10, 10)

        self.model = nn.Sequential(
            ## put some code here ##
        )

    def forward(self, x, labels):
        x = x.view(x.size(0), 784)
        c = self.label_emb(labels)
        ## Change ?? in the next line
        x = torch.cat([??, ??], 1)
        ##
        out = self.model(x)
        return out.squeeze()

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()

        self.label_emb = nn.Embedding(10, 10)

        self.model = nn.Sequential(
            ## put some code here ##
        )

    def forward(self, z, labels):
        z = z.view(z.size(0), 100)
        c = self.label_emb(labels)
        ## Change ?? in the next line
        x = torch.cat([??, ??], 1)
        ##
        out = self.model(x)
        return out.view(x.size(0), 28, 28)

Обучим модель.

In [ ]:
generator = Generator().cuda()
discriminator = Discriminator().cuda()

In [ ]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-4)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=1e-4)

In [ ]:
def discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels):
    d_optimizer.zero_grad()

    # train with real images
    real_validity = discriminator(real_images, labels)
    real_loss = criterion(real_validity, torch.ones(batch_size).cuda())

    # train with fake images
    z = torch.randn(batch_size, 100).cuda()
    fake_labels = torch.LongTensor(np.random.randint(0, 10, batch_size)).cuda()
    fake_images = generator(z, fake_labels)
    fake_validity = discriminator(fake_images, fake_labels)
    fake_loss = criterion(fake_validity, torch.zeros(batch_size).cuda())

    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss.data

In [ ]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion):
    g_optimizer.zero_grad()
    z = torch.randn(batch_size, 100).cuda()
    fake_labels = torch.LongTensor(np.random.randint(0, 10, batch_size)).cuda()
    fake_images = generator(z, fake_labels)
    validity = discriminator(fake_images, fake_labels)
    g_loss = criterion(validity, torch.ones(batch_size).cuda())
    g_loss.backward()
    g_optimizer.step()
    return g_loss.data

In [ ]:
num_epochs = 30
n_critic = 5
display_step = 300
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch}...')
    for i, (images, labels) in enumerate(data_loader):
        real_images = images.cuda()
        labels = labels.cuda()
        generator.train()
        batch_size = real_images.size(0)
        d_loss = discriminator_train_step(len(real_images), discriminator,
                                          generator, d_optimizer, criterion,
                                          real_images, labels)


        g_loss = generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion)

    generator.eval()
    print(f'g_loss: {g_loss:.4f}, d_loss: {d_loss:.4f}')
    z = torch.randn(9, 100).cuda()
    labels = torch.LongTensor(np.arange(9)).cuda()
    sample_images = generator(z, labels).unsqueeze(1).data.cpu()
    grid = make_grid(sample_images, nrow=3, normalize=True).permute(1,2,0).numpy()
    plt.imshow(grid)
    plt.show()

Попробуем сгенерировать по несколько предметов каждого класса.

In [ ]:
z = torch.randn(100, 100).cuda()
labels = torch.LongTensor([i for _ in range(10) for i in range(10)]).cuda()
sample_images = generator(z, labels).unsqueeze(1).data.cpu()
grid = make_grid(sample_images, nrow=10, normalize=True).permute(1,2,0).numpy()
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(grid)
_ = plt.yticks([])
_ = plt.xticks(np.arange(15, 300, 30), ['T-Shirt', 'Trouser', 'Pullover', 'Dress',\
                                        'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag',\
                                        'Ankle boot'], rotation=45, fontsize=20)